In [1]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
import numpy as np
import sklearn
import re
import requests

In [2]:
base_url = 'https://www.timeanddate.com/weather/usa/new-york/historic?month=1&year=2020'
uuu='https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200206&month=2&year=2020&json=1'

In [3]:
headers = {
    'accept':'*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36'
}
data={'n': 'usa/new-york',
          'mode': 'historic',
          'hd': 20200110,
          'month': 1,
          'year': '2020'}

In [4]:
session = requests.Session()
request = session.get(new_urls[0], headers = headers)
print(request)

NameError: name 'new_urls' is not defined

In [22]:
data = request.text

In [24]:
data

'[{c:[{h:"00:27<br><span class=\\"smaller soft\\">Sat, 1 Feb<\\/span>"},{s:"wt-ic",h:"<img class=mtt title=\\"Overcast.\\" src=\\"\\/\\/c.tadst.com\\/gfx\\/w\\/svg\\/wt-17.svg\\" width=40 height=40>"},{h:"4&nbsp;°C"},{s:"small",h:"Overcast."},{s:"sep",h:"7 km\\/h"},{h:"<span class=\\"comp sa22\\" title=\\"Wind blowing from 60° East-northeast to West-southwest\\">↑<\\/span>"},{h:"86%"},{s:"sep",h:"1021 mbar"},{h:"16&nbsp;km"}]},{c:[{h:"00:51"},{s:"wt-ic",h:"<img class=mtt title=\\"Overcast.\\" src=\\"\\/\\/c.tadst.com\\/gfx\\/w\\/svg\\/wt-17.svg\\" width=40 height=40>"},{h:"4&nbsp;°C"},{s:"small",h:"Overcast."},{s:"sep",h:"7 km\\/h"},{h:"<span class=\\"comp sa20\\" title=\\"Wind blowing from 50° Northeast to Southwest\\">↑<\\/span>"},{h:"86%"},{s:"sep",h:"1020 mbar"},{h:"14&nbsp;km"}]},{c:[{h:"01:51"},{s:"wt-ic",h:"<img class=mtt title=\\"Overcast.\\" src=\\"\\/\\/c.tadst.com\\/gfx\\/w\\/svg\\/wt-17.svg\\" width=40 height=40>"},{h:"4&nbsp;°C"},{s:"small",h:"Overcast."},{s:"sep",h:"6 km\

In [279]:
data_h=[]
for i in range(len(data)):
    if(data[i]=='h' and data[i+1]==':'):
        string=''
        i=i+3
        while(data[i]!='"'):
            string+=data[i]
            i+=1
        data_h.append(string)
    

In [204]:
data_h

['00:07<br><span class=\\',
 '<img class=mtt title=\\',
 '12&nbsp;°C',
 'Mostly cloudy.',
 '15 km\\/h',
 '<span class=\\',
 '83%',
 '1009 mbar',
 '14&nbsp;km',
 '00:51',
 '<img class=mtt title=\\',
 '9&nbsp;°C',
 'Overcast.',
 '13 km\\/h',
 '<span class=\\',
 '80%',
 '1009 mbar',
 '16&nbsp;km',
 '01:23',
 '<img class=mtt title=\\',
 '7&nbsp;°C',
 'Light rain. Fog.',
 '13 km\\/h',
 '<span class=\\',
 '86%',
 '1010 mbar',
 '10&nbsp;km',
 '01:51',
 '<img class=mtt title=\\',
 '7&nbsp;°C',
 'Overcast.',
 '13 km\\/h',
 '<span class=\\',
 '82%',
 '1010 mbar',
 '16&nbsp;km',
 '02:51',
 '<img class=mtt title=\\',
 '6&nbsp;°C',
 'Overcast.',
 '17 km\\/h',
 '<span class=\\',
 '79%',
 '1011 mbar',
 '16&nbsp;km',
 '02:59',
 '<img class=mtt title=\\',
 '6&nbsp;°C',
 'Overcast.',
 '17 km\\/h',
 '<span class=\\',
 '79%',
 '1011 mbar',
 '16&nbsp;km',
 '03:51',
 '<img class=mtt title=\\',
 '4&nbsp;°C',
 'Fog.',
 '15 km\\/h',
 '<span class=\\',
 '86%',
 '1011 mbar',
 '8&nbsp;km',
 '04:12',
 '<img class=

In [161]:
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
time=[]
for title in data_h:
    if(re.match('^[0-9].*C$', title)):
        temp.append(title)
    if(re.match('^[0-9].*/h$', title) or re.match('No wind',title) or re.match('N/A',title)):#--
        wind.append(title)
    if(re.match('^[0-9].*km$', title)):
        vis.append(title)
    if(re.match('..%', title) or re.match('.%', title)):
        hum.append(title)
    if(re.match('^[A-Z]', title) and title!='Weather by CustomWeather, © 2022' and title!='No wind' and title!='N/A'):
        weather.append(title)
    if(re.match('^[0-9].*mbar$',title)):
        barom.append(title)
    if(re.match('^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$',title) or re.match('([0-1]?[0-9]|2[0-3]):[0-5][0-9]',title)):
        time.append(title)

In [178]:
time[0]=time[0][:5]

In [158]:
new_temp=[]
for i in temp:
    i = i.replace('&nbsp;',' ')
    new_temp.append(i)

In [167]:
new_wind=[]
for i in wind:
    i = i.replace('\\','')
    new_wind.append(i)

In [170]:
new_vis=[]
for i in vis:
    i = i.replace('&nbsp;',' ')
    new_vis.append(i)

In [179]:
df_202002 = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility'])
df_202002['Day']='20200110'
df_202002['Time']=time
df_202002['Temp']=new_temp
df_202002['Weather']=weather
df_202002['Wind']=new_wind
df_202002['Humidity']=hum
df_202002['Barometer']=barom
df_202002['Visibility']=new_vis 

In [180]:
df_202002

,Day,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility
0,NaN,02:06,2 °C,Rain. Fog.,7 km/h,92%,1012 mbar,3 km
1,NaN,02:51,2 °C,Light rain. Fog.,9 km/h,92%,1010 mbar,6 km
2,NaN,06:19,4 °C,Light rain. Fog.,13 km/h,93%,1007 mbar,5 km
3,NaN,06:51,4 °C,Fog.,11 km/h,96%,1005 mbar,10 km
4,NaN,07:51,4 °C,Light rain. Fog.,11 km/h,93%,1005 mbar,10 km
5,NaN,08:51,5 °C,Fog.,11 km/h,93%,1005 mbar,5 km
6,NaN,09:07,5 °C,Fog.,11 km/h,93%,1006 mbar,4 km
7,NaN,09:35,5 °C,Fog.,17 km/h,93%,1006 mbar,3 km
8,NaN,09:51,5 °C,Fog.,20 km/h,93%,1004 mbar,3 km
9,NaN,10:05,5 °C,Fog.,7 km/h,93%,1005 mbar,5 km


In [324]:
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
time=[]
data_h=[]
day=[]
df_202002 = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility'])
for k in range(len(new_urls)):
    session = requests.Session()
    request = session.get(new_urls[k], headers = headers)
    data = request.text   
    for i in range(len(data)):
        if(data[i]=='h' and data[i+1]==':'):
            string=''
            i=i+3
            while(data[i]!='"'):
                string+=data[i]
                i+=1
            data_h.append(string)
    if request.status_code == 200:
        for title in data_h:
            if(re.match('^([0-1]?[0-9]|2[0-3]):[0-5][0-9]$',title) or re.match('([0-1]?[0-9]|2[0-3]):[0-5][0-9]',title)):
                time.append(title)
                day.append(all_days[k])
            if(re.match('^[0-9].*C$', title)):
                temp.append(title)
            if(re.match('^[A-Z]', title) and title!='Weather by CustomWeather, © 2022' and title!='No wind' and title!='N\\/A'):
                weather.append(title)
            if(re.match('^[0-9].*/h$', title) or re.match('No wind',title) or title=='N\\/A'):
                wind.append(title)
            if(re.match('..%', title) or re.match('.%', title)):
                hum.append(title)
            if(re.match('^[0-9].*mbar$',title) or title=='N\\/A'):
                barom.append(title)
            if(re.match('^[0-9].*km$', title)):
                vis.append(title)
            
            
        df_202002['Day']=day
        df_202002['Time']=time
        df_202002['Temp']=temp
        df_202002['Weather']=weather
        df_202002['Wind']=wind
        df_202002['Humidity']=hum
        df_202002['Barometer']=barom
        df_202002['Visibility']=vis 
        print('Okay = ',k)

new_temp=[]
for i in temp:
    i = i.replace('&nbsp;',' ')
    new_temp.append(i) 
    
new_wind=[]
for i in wind:
    i = i.replace('\\','')
    new_wind.append(i)
    
new_vis=[]
for i in vis:
    i = i.replace('&nbsp;',' ')
    new_vis.append(i)

new_time=[]
for i in time:
    if(len(i)>6):
        new_time.append(i[:5])
    else:
        new_time.append(i)
                    




ValueError: Length of values (33) does not match length of index (32)

In [325]:
wind

['7 km\\/h',
 '7 km\\/h',
 '6 km\\/h',
 '9 km\\/h',
 '6 km\\/h',
 '6 km\\/h',
 '6 km\\/h',
 'No wind',
 'No wind',
 'No wind',
 'No wind',
 'No wind',
 '7 km\\/h',
 'No wind',
 'No wind',
 'No wind',
 '6 km\\/h',
 'No wind',
 '6 km\\/h',
 'No wind',
 '6 km\\/h',
 'N\\/A',
 'No wind',
 '6 km\\/h',
 'No wind',
 'No wind',
 '6 km\\/h',
 'No wind',
 '6 km\\/h',
 '7 km\\/h',
 '6 km\\/h',
 '6 km\\/h',
 'No wind']

In [45]:
import scrapy
response = scrapy.http.HtmlResponse(url=uuu)
response

<200 https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200206&month=2&year=2020&json=1>

In [28]:
data_c=[]
for i in range(len(data)):
    if(data[i]=='['):
        j=0
        string=''
        while(data[i]!=']'):
            string+=data[i]
            i+=1
        data_c.append(string)

In [45]:
data_c_h=[]
for i in range(len(data_c[0])):
    if(data_c[0][i]=='h' and data_c[0][i+1]==':'):
        string=''
        i=i+3
        while(data_c[0][i]!='"'):
            string+=data_c[0][i]
            i+=1
        data_c_h.append(string)


In [46]:
data_c_h.pop(1)
data_c_h.pop(4)
data_c_h

['00:27<br><span class=\\',
 '4&nbsp;°C',
 'Overcast.',
 '7 km\\/h',
 '86%',
 '1021 mbar',
 '16&nbsp;km']

In [71]:
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
time=[]
data_h=[]
day=[]
data_c=[]
df_202002 = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility'])
for k in range(len(new_urls)):
    session = requests.Session()
    request = session.get(new_urls[k], headers = headers)
    data = request.text   
    for i in range(len(data)):
        if(data[i]=='['):
            j=0
            string=''
            while(data[i]!=']'):
                string+=data[i]
                i+=1
            data_h.append(string)
    for j in range(len(data_h)):
        data_c_h=[]
        data_c_h.append(all_days[k])
        for i in range(len(data_h[j])): 
            if(data_h[j][i]=='h' and data_h[j][i+1]==':'):
                string=''
                i=i+3
                while(data_h[j][i]!='"'):
                    string+=data_h[j][i]
                    i+=1
                data_c_h.append(string)
        if(len(data_c_h)!=0): 
            data_c_h.pop(2)
            data_c_h.pop(5)
            data_c.append(data_c_h)
        


In [72]:
data_c

[['20200201',
  '00:27<br><span class=\\',
  '4&nbsp;°C',
  'Overcast.',
  '7 km\\/h',
  '86%',
  '1021 mbar',
  '16&nbsp;km'],
 ['20200201',
  '00:27<br><span class=\\',
  '4&nbsp;°C',
  'Overcast.',
  '7 km\\/h',
  '86%',
  '1021 mbar',
  '16&nbsp;km'],
 ['20200201',
  '00:51',
  '4&nbsp;°C',
  'Overcast.',
  '7 km\\/h',
  '86%',
  '1020 mbar',
  '14&nbsp;km'],
 ['20200201',
  '01:51',
  '4&nbsp;°C',
  'Overcast.',
  '6 km\\/h',
  '86%',
  '1019 mbar',
  '16&nbsp;km'],
 ['20200201',
  '02:51',
  '4&nbsp;°C',
  'Overcast.',
  '9 km\\/h',
  '86%',
  '1018 mbar',
  '16&nbsp;km'],
 ['20200201',
  '03:51',
  '3&nbsp;°C',
  'Fog.',
  '6 km\\/h',
  '89%',
  '1018 mbar',
  '8&nbsp;km'],
 ['20200201',
  '04:51',
  '3&nbsp;°C',
  'Fog.',
  '6 km\\/h',
  '90%',
  '1017 mbar',
  '8&nbsp;km'],
 ['20200201',
  '05:51',
  '4&nbsp;°C',
  'Fog.',
  '6 km\\/h',
  '86%',
  '1016 mbar',
  '8&nbsp;km'],
 ['20200201',
  '06:51',
  '4&nbsp;°C',
  'Fog.',
  'No wind',
  '89%',
  '1016 mbar',
  '8&nbsp;km'],

In [73]:
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
time=[]
day=[]
new_weather=[]
df_202002 = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility'])

for i in range(len(data_c)):
    weather=[]
    for j in data_c[i]:
        j = j.replace('&nbsp;',' ')
        j = j.replace('\\','')
        j = j.replace('<br><span class=','')
        weather.append(j)
    new_weather.append(weather)
        
        

In [74]:
new_weather

[['20200201',
  '00:27',
  '4 °C',
  'Overcast.',
  '7 km/h',
  '86%',
  '1021 mbar',
  '16 km'],
 ['20200201',
  '00:27',
  '4 °C',
  'Overcast.',
  '7 km/h',
  '86%',
  '1021 mbar',
  '16 km'],
 ['20200201',
  '00:51',
  '4 °C',
  'Overcast.',
  '7 km/h',
  '86%',
  '1020 mbar',
  '14 km'],
 ['20200201',
  '01:51',
  '4 °C',
  'Overcast.',
  '6 km/h',
  '86%',
  '1019 mbar',
  '16 km'],
 ['20200201',
  '02:51',
  '4 °C',
  'Overcast.',
  '9 km/h',
  '86%',
  '1018 mbar',
  '16 km'],
 ['20200201', '03:51', '3 °C', 'Fog.', '6 km/h', '89%', '1018 mbar', '8 km'],
 ['20200201', '04:51', '3 °C', 'Fog.', '6 km/h', '90%', '1017 mbar', '8 km'],
 ['20200201', '05:51', '4 °C', 'Fog.', '6 km/h', '86%', '1016 mbar', '8 km'],
 ['20200201', '06:51', '4 °C', 'Fog.', 'No wind', '89%', '1016 mbar', '8 km'],
 ['20200201', '07:16', '4 °C', 'Fog.', 'No wind', '89%', '1017 mbar', '8 km'],
 ['20200201', '08:13', '4 °C', 'Fog.', 'No wind', '89%', '1017 mbar', '6 km'],
 ['20200201', '08:19', '4 °C', 'Fog.', 

In [75]:
import pandas as pd  

df_202002 = pd.DataFrame(new_weather, columns =['Day',
                                                'Time',
                                                 'Temp',
                                                 'Weather',
                                                 'Wind',
                                                 'Humidity',
                                                 'Barometer',
                                                 'Visibility']) 

In [82]:
df_202002['Barometer'].value_counts()

1010 mbar    718
1017 mbar    565
1019 mbar    522
1011 mbar    487
1003 mbar    470
1021 mbar    467
1009 mbar    443
1020 mbar    440
1016 mbar    435
1022 mbar    435
1012 mbar    427
1008 mbar    411
1005 mbar    408
1001 mbar    385
1018 mbar    365
1031 mbar    329
1023 mbar    296
1029 mbar    290
1014 mbar    284
1006 mbar    268
1007 mbar    244
1030 mbar    238
1013 mbar    232
1015 mbar    232
1024 mbar    229
1028 mbar    224
1025 mbar    222
1004 mbar    218
1000 mbar    217
1002 mbar    216
1026 mbar    194
1027 mbar    191
998 mbar     158
1032 mbar    144
1033 mbar    142
999 mbar     129
995 mbar      99
996 mbar      96
1037 mbar     90
1036 mbar     77
985 mbar      69
977 mbar      69
1038 mbar     60
1034 mbar     58
997 mbar      50
992 mbar      49
976 mbar      46
990 mbar      46
N/A           39
1035 mbar     31
994 mbar      26
980 mbar      23
988 mbar      23
991 mbar      23
986 mbar      23
982 mbar      23
979 mbar      23
983 mbar      23
993 mbar      

In [46]:
soup = bs(request.content,'lxml')
soup

<html><body><p>[{c:[{h:"02:06<br/><span class='\"smaller' soft="">Thu, 6 Feb"},{s:"wt-ic",h:"<img class="mtt" fog.="" height="40" src='\"\/\/c.tadst.com\/gfx\/w\/svg\/wt-19.svg\"' title='\"Rain.' width="40"/>"},{h:"2 °C"},{s:"small",h:"Rain. Fog."},{s:"sep",h:"7 km\/h"},{h:"<span blowing="" class='\"comp' from="" north="" sa16="" south="" title='\"Wind' to="">↑"},{h:"92%"},{s:"sep",h:"1012 mbar"},{h:"3 km"}]},{c:[{h:"02:51"},{s:"wt-ic",h:"<img class="mtt" fog.="" height="40" rain.="" src='\"\/\/c.tadst.com\/gfx\/w\/svg\/wt-34.svg\"' title='\"Light' width="40"/>"},{h:"2 °C"},{s:"small",h:"Light rain. Fog."},{s:"sep",h:"9 km\/h"},{h:"<span blowing="" class='\"comp' east="" from="" sa24="" title='\"Wind' to="" west="">↑"},{h:"92%"},{s:"sep",h:"1010 mbar"},{h:"6 km"}]},{c:[{h:"06:19"},{s:"wt-ic",h:"<img class="mtt" fog.="" height="40" rain.="" src='\"\/\/c.tadst.com\/gfx\/w\/svg\/wt-34.svg\"' title='\"Light' width="40"/>"},{h:"4 °C"},{s:"small",h:"Light rain. Fog."},{s:"sep",h:"13 km\/h"},

In [48]:
block = soup.find('div',class_ = 'weatherLinks')
block

In [50]:
all_day = block.find_all(href=True)
all_day[0].get('href')
#for i in range all_day:
#    print

'/weather/usa/new-york/historic?hd=20200101'

In [6]:
urls=[]
#urls.append(base_url)
count = 13
for i in range(count-1):
            url = f'https://www.timeanddate.com/weather/usa/new-york/historic?month={i+1}&year=2020'
            urls.append(url)

In [7]:
urls

['https://www.timeanddate.com/weather/usa/new-york/historic?month=1&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=2&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=3&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=4&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=5&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=6&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=7&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=8&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=9&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=10&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=11&year=2020',
 'https://www.timeanddate.com/weather/usa/new-york/historic?month=12&year=2020']

In [22]:
headers2 = {
    'accept':'*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
}
data={
    'n': 'usa/new-york',
    'mode': 'historic',
    'hd': '20200102',
    'month': '1',
    'year': '2020',
    'json': '1'
    }


In [24]:
session = requests.Session()
request2 = session.get(base_url, headers = headers2,data=data)
print(request2.response)

<Response [200]>


In [9]:
soup = bs(request.content,'lxml')
soup

<!DOCTYPE html>
<!--
scripts and programs that download content transparent to the user are not allowed without permission
--><html lang="en"><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><title>Weather in January 2020 in New York, New York, USA</title><meta content="Weather reports from January 2020 in New York, New York, USA with highs and lows" name="description"/><meta content="https://www.timeanddate.com/scripts/cityog.php?title=Past%20Weather%20in&amp;tint=0x007b7a&amp;city=New%20York&amp;state=New%20York&amp;country=USA&amp;image=new-york1" property="og:image"/><meta content="1366" property="og:image:width"/><meta content="738" property="og:image:height"/><meta content="website" property="og:type"/><style>
@font-face{font-family:iconfont;src:url("/common/fonts/iconfont.woff2?v8") format("woff2"),url("/common/fonts/iconfont.woff?v8") format("woff"),url("/common/fonts/iconfont.ttf?v8") format("truetype"),url("/common/fonts/iconfont.svg?v8#iconfont") for

In [10]:
block2 = soup.find('div',class_ = 'tb-scroll')
print(block2)

<div class="tb-scroll"><span>Scroll right to see more</span><table class="zebra tb-wt fw va-m tb-hover" id="wt-his"><thead><tr class="bg-wt"><th> </th><th class="sep" colspan="3">Conditions</th><th class="sep" colspan="3">Comfort</th><th class="sep"> </th><th> </th></tr><tr><th>Time</th><th class="sep"> </th><th>Temp</th><th>Weather</th><th class="sep">Wind</th><th> </th><th>Humidity</th><th class="sep">Barometer</th><th>Visibility</th></tr></thead><tbody><tr><th>00:51<br/><span class="smaller soft">Wed, 1 Jan</span></th><td class="wt-ic"><img class="mtt" height="40" src="//c.tadst.com/gfx/w/svg/wt-17.svg" title="Mostly cloudy." width="40"/></td><td>4 °C</td><td class="small">Mostly cloudy.</td><td class="sep">13 km/h</td><td><span class="comp sa8" title="Wind blowing from 270° West to East">↑</span></td><td>58%</td><td class="sep">1004 mbar</td><td>16 km</td></tr><tr><th>01:51</th><td class="wt-ic"><img class="mtt" height="40" src="//c.tadst.com/gfx/w/svg/wt-17.svg" title="Mostly clou

In [11]:
all_hour = block2.find_all('tr', attrs = {'class':None})
all_hour[0]

<tr><th>Time</th><th class="sep"> </th><th>Temp</th><th>Weather</th><th class="sep">Wind</th><th> </th><th>Humidity</th><th class="sep">Barometer</th><th>Visibility</th></tr>

In [52]:
hed=[]
for i in all_hour[0].find_all('th'):
    title = i.text
    hed.append(title)

In [53]:
hed

['Time',
 '\xa0',
 'Temp',
 'Weather',
 'Wind',
 '\xa0',
 'Humidity',
 'Barometer',
 'Visibility']

In [269]:
all_hour

[<tr><th>Time</th><th class="sep"> </th><th>Temp</th><th>Weather</th><th class="sep">Wind</th><th> </th><th>Humidity</th><th class="sep">Barometer</th><th>Visibility</th></tr>,
 <tr><th>00:51<br/><span class="smaller soft">Wed, 1 Jan</span></th><td class="wt-ic"><img class="mtt" height="40" src="//c.tadst.com/gfx/w/svg/wt-17.svg" title="Mostly cloudy." width="40"/></td><td>4 °C</td><td class="small">Mostly cloudy.</td><td class="sep">13 km/h</td><td><span class="comp sa8" title="Wind blowing from 270° West to East">↑</span></td><td>58%</td><td class="sep">1004 mbar</td><td>16 km</td></tr>,
 <tr><th>01:51</th><td class="wt-ic"><img class="mtt" height="40" src="//c.tadst.com/gfx/w/svg/wt-17.svg" title="Mostly cloudy." width="40"/></td><td>4 °C</td><td class="small">Mostly cloudy.</td><td class="sep">13 km/h</td><td><span class="comp sa8" title="Wind blowing from 280° West to East">↑</span></td><td>61%</td><td class="sep">1004 mbar</td><td>16 km</td></tr>,
 <tr><th>02:51</th><td class="wt

In [191]:
time=[]
for j in all_hour:
    for i in j.find_all('th'):
        title = i.text
        time.append(title)

In [62]:
weath=[]
for j in all_hour:
    for i in j.find_all('td'):
        title = i.text
        weath.append(title)

In [65]:
weath


['',
 '4\xa0°C',
 'Mostly cloudy.',
 '13 km/h',
 '↑',
 '58%',
 '1004 mbar',
 '16\xa0km',
 '',
 '4\xa0°C',
 'Mostly cloudy.',
 '13 km/h',
 '↑',
 '61%',
 '1004 mbar',
 '16\xa0km',
 '',
 '4\xa0°C',
 'Overcast.',
 '22 km/h',
 '↑',
 '60%',
 '1004 mbar',
 '16\xa0km',
 '',
 '4\xa0°C',
 'Overcast.',
 '19 km/h',
 '↑',
 '55%',
 '1005 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '9 km/h',
 '↑',
 '55%',
 '1005 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '6 km/h',
 '↑',
 '54%',
 '1005 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '13 km/h',
 '↑',
 '62%',
 '1006 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '13 km/h',
 '↑',
 '60%',
 '1006 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '19 km/h',
 '↑',
 '54%',
 '1007 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '9 km/h',
 '↑',
 '53%',
 '1008 mbar',
 '16\xa0km',
 '',
 '3\xa0°C',
 'Overcast.',
 '19 km/h',
 '↑',
 '46%',
 '1007 mbar',
 '16\xa0km',
 '',
 '4\xa0°C',
 'Partly sunny.',
 '19 km/h',
 '↑',
 '43%',
 '1007 m

In [264]:
import pandas as pd  

df = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility']) 

In [196]:
df

,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility


In [197]:
df['Time']= time[9:]

In [198]:
df['day']=20200101

In [84]:
import re
<td style="width: 86px;">14&nbsp;°C</td>

In [201]:
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
for j in block2.find_all('tr', attrs = {'class':None})[1:]:
    for i in j.find_all('td'):
        title = i.text
         if(re.match('^[0-9].*C$', title)):
            temp.append(title)
        if(re.match('^[0-9].*km/h$', title) or re.match('No wind',title) or re.match('N/A',title)):
            wind.append(title)
        if(re.match('^[0-9]..km$', title)):
            vis.append(title)
        if(re.match('..%', title) or re.match('.%', title)):
            hum.append(title)
        if(re.match('^[A-Z]', title) and title!='Weather by CustomWeather, © 2022' and title!='No wind' and title!='N/A'):
            weather.append(title)
        if(re.match('^[0-9].*mbar$',title)):
            barom.append(title)
            


In [202]:
len(temp)

24

In [203]:
df['Temp']=temp
df['Weather']=weather
df['Wind']=wind
df['Humidity']=hum
df['Barometer']=barom
df['Visibility']=vis 

In [204]:
df

,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility,day
0,"00:51Wed, 1 Jan",4 °C,Mostly cloudy.,13 km/h,58%,1004 mbar,16 km,20200101
1,01:51,4 °C,Mostly cloudy.,13 km/h,61%,1004 mbar,16 km,20200101
2,02:51,4 °C,Overcast.,22 km/h,60%,1004 mbar,16 km,20200101
3,03:51,4 °C,Overcast.,19 km/h,55%,1005 mbar,16 km,20200101
4,04:51,3 °C,Overcast.,9 km/h,55%,1005 mbar,16 km,20200101
5,05:51,3 °C,Overcast.,6 km/h,54%,1005 mbar,16 km,20200101
6,06:51,3 °C,Overcast.,13 km/h,62%,1006 mbar,16 km,20200101
7,07:51,3 °C,Overcast.,13 km/h,60%,1006 mbar,16 km,20200101
8,08:51,3 °C,Overcast.,19 km/h,54%,1007 mbar,16 km,20200101
9,09:51,3 °C,Overcast.,9 km/h,53%,1008 mbar,16 km,20200101


In [ ]:
pd.concat([df2, df])

In [209]:
headers = {
    'accept':'*/*',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.74 Safari/537.36',
}


In [10]:
all_days=[]
print(date_d[0].strftime("%Y%m%d"))

20200101


In [12]:
date_d = pd.date_range(
    start='20200201',
    periods=29,
    freq='D'
)

In [13]:
all_days=[]
for i in date_d:
    all_days.append(i.strftime("%Y%m%d"))

In [14]:
all_days

['20200201',
 '20200202',
 '20200203',
 '20200204',
 '20200205',
 '20200206',
 '20200207',
 '20200208',
 '20200209',
 '20200210',
 '20200211',
 '20200212',
 '20200213',
 '20200214',
 '20200215',
 '20200216',
 '20200217',
 '20200218',
 '20200219',
 '20200220',
 '20200221',
 '20200222',
 '20200223',
 '20200224',
 '20200225',
 '20200226',
 '20200227',
 '20200228',
 '20200229']

In [15]:
new_url='https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200102&month=1&year=2020&json=1'

In [16]:
new_urls=[]
for i in all_days:
    new_urls.append(f'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd={i}&month=2&year=2020&json=1')

In [17]:
new_urls

['https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200201&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200202&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200203&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200204&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200205&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200206&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200207&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.php?n=usa/new-york&mode=historic&hd=20200208&month=2&year=2020&json=1',
 'https://www.timeanddate.com/scripts/cityajax.p

In [41]:
day=[]
time=[]
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
for k in all_days:
    data={'n': 'usa/new-york',
          'mode': 'historic',
          'hd': k,
          'month': 2,
          'year': '2020'}
    session = requests.Session()
    request = session.get(urls[1], headers = headers,data=data)
    if request.status_code == 200:
        soup = bs(request.content,'lxml')
        block = soup.find('div',class_ = 'tb-scroll')
        all_hour = block.find_all('tr', attrs = {'class':None})
        for j in all_hour:
            for i in j.find_all('th'):
                title = i.text
                if(re.match('^[0-9]', title)):
                    time.append(title)
                    day.append(k)
       
        for j in block.find_all('tr', attrs = {'class':None})[1:]:
            for i in j.find_all('td'):
                title = i.text
                if(re.match('^[0-9].*C$', title)):
                    temp.append(title)
                if(re.match('^[0-9].*km/h$', title) or re.match('No wind',title) or re.match('N/A',title)):
                    wind.append(title)
                if(re.match('^[0-9]..km$', title)):
                    vis.append(title)
                if(re.match('..%', title) or re.match('.%', title)):
                    hum.append(title)
                if(re.match('^[A-Z]', title) and title!='Weather by CustomWeather, © 2022' and title!='No wind' and title!='N/A'):
                    weather.append(title)
                if(re.match('^[0-9].*mbar$',title)):
                    barom.append(title)
            
i=0
while i < (len(time)):
    time[i]='00:51'
    i=i+24
    
                    
df_202002 = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility'])
df_202002['Day']=day
df_202002['Time']=time
df_202002['Temp']=temp
df_202002['Weather']=weather
df_202002['Wind']=wind
df_202002['Humidity']=hum
df_202002['Barometer']=barom
df_202002['Visibility']=vis 

AttributeError: 'NoneType' object has no attribute 'find_all'

In [28]:
day=[]
time=[]
temp=[]
wind=[]
hum=[]
weather=[]
barom = []
vis = []
data={'n': 'usa/new-york',
          'mode': 'historic',
          'hd': 20200110,
          'month': 1,
          'year': '2020'}
session = requests.Session()
request = session.get(urls[0], headers = headers,data=data)
if request.status_code == 200:
    soup = bs(request.content,'lxml')
    block = soup.find('div',class_ = 'tb-scroll')
    all_hour = block.find_all('tr', attrs = {'class':None})
    for j in all_hour:
        for i in j.find_all('th'):
            title = i.text
            if(re.match('^[0-9]', title)):
                time.append(title)
                day.append('20200211')
       
    for j in block.find_all('tr', attrs = {'class':None})[1:]:
        for i in j.find_all('td'):
            title = i.text
            if(re.match('^[0-9].*C$', title)):
                temp.append(title)
            if(re.match('^[0-9].*km/h$', title) or re.match('No wind',title) or re.match('N/A',title)):
                wind.append(title)
            if(re.match('^[0-9]..km$', title)):
                vis.append(title)
            if(re.match('..%', title) or re.match('.%', title)):
                hum.append(title)
            if(re.match('^[A-Z]', title) and title!='Weather by CustomWeather, © 2022' and title!='No wind' and title!='N/A'):
                weather.append(title)
            if(re.match('^[0-9].*mbar$',title)):
                barom.append(title)
            
i=0
while i < (len(time)):
    time[i]='00:51'
    i=i+24
    
                    
df_202002 = pd.DataFrame(columns =['Day',
                            'Time',
                             'Temp',
                             'Weather',
                             'Wind',
                             'Humidity',
                             'Barometer',
                             'Visibility'])
df_202002['Day']=day
df_202002['Time']=time
df_202002['Temp']=temp
df_202002['Weather']=weather
df_202002['Wind']=wind
df_202002['Humidity']=hum
df_202002['Barometer']=barom
df_202002['Visibility']=vis 

In [31]:
wind

['13 km/h',
 '13 km/h',
 '22 km/h',
 '19 km/h',
 '9 km/h',
 '6 km/h',
 '13 km/h',
 '13 km/h',
 '19 km/h',
 '9 km/h',
 '19 km/h',
 '19 km/h',
 '13 km/h',
 '15 km/h',
 '19 km/h',
 '17 km/h',
 '11 km/h',
 '11 km/h',
 '15 km/h',
 '15 km/h',
 '15 km/h',
 '9 km/h',
 '13 km/h',
 '7 km/h']

In [32]:
df_202002

,Day,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility
0,20200211,00:51,4 °C,Mostly cloudy.,13 km/h,58%,1004 mbar,16 km
1,20200211,01:51,4 °C,Mostly cloudy.,13 km/h,61%,1004 mbar,16 km
2,20200211,02:51,4 °C,Overcast.,22 km/h,60%,1004 mbar,16 km
3,20200211,03:51,4 °C,Overcast.,19 km/h,55%,1005 mbar,16 km
4,20200211,04:51,3 °C,Overcast.,9 km/h,55%,1005 mbar,16 km
5,20200211,05:51,3 °C,Overcast.,6 km/h,54%,1005 mbar,16 km
6,20200211,06:51,3 °C,Overcast.,13 km/h,62%,1006 mbar,16 km
7,20200211,07:51,3 °C,Overcast.,13 km/h,60%,1006 mbar,16 km
8,20200211,08:51,3 °C,Overcast.,19 km/h,54%,1007 mbar,16 km
9,20200211,09:51,3 °C,Overcast.,9 km/h,53%,1008 mbar,16 km


In [31]:
i=0
while i < (len(time)):
    time[i]='00:51'
    i=i+24

In [32]:
time[48]

'00:51'

In [284]:
time=[]
for j in all_hour:
    for i in j.find_all('th'):
        title = i.text
        if(re.match('^[0-9]', title)):
            time.append(title)

In [34]:
df_202001['Time']=time



In [47]:
df_202001

,Day,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility
0,20200101,00:51,4 °C,Mostly cloudy.,13 km/h,58%,1004 mbar,16 km
1,20200101,01:51,4 °C,Mostly cloudy.,13 km/h,61%,1004 mbar,16 km
2,20200101,02:51,4 °C,Overcast.,22 km/h,60%,1004 mbar,16 km
3,20200101,03:51,4 °C,Overcast.,19 km/h,55%,1005 mbar,16 km
4,20200101,04:51,3 °C,Overcast.,9 km/h,55%,1005 mbar,16 km
...,...,...,...,...,...,...,...,...
739,20200131,19:51,2 °C,Passing clouds.,15 km/h,50%,1011 mbar,16 km
740,20200131,20:51,2 °C,Clear.,15 km/h,52%,1012 mbar,16 km
741,20200131,21:51,2 °C,Clear.,9 km/h,52%,1012 mbar,16 km
742,20200131,22:51,2 °C,Clear.,13 km/h,52%,1012 mbar,16 km


In [43]:
df_202002

,Day,Time,Temp,Weather,Wind,Humidity,Barometer,Visibility
0,20200201,00:51,4 °C,Mostly cloudy.,13 km/h,58%,1004 mbar,16 km
1,20200201,01:51,4 °C,Mostly cloudy.,13 km/h,61%,1004 mbar,16 km
2,20200201,02:51,4 °C,Overcast.,22 km/h,60%,1004 mbar,16 km
3,20200201,03:51,4 °C,Overcast.,19 km/h,55%,1005 mbar,16 km
4,20200201,04:51,3 °C,Overcast.,9 km/h,55%,1005 mbar,16 km
...,...,...,...,...,...,...,...,...
691,20200229,19:51,2 °C,Passing clouds.,15 km/h,50%,1011 mbar,16 km
692,20200229,20:51,2 °C,Clear.,15 km/h,52%,1012 mbar,16 km
693,20200229,21:51,2 °C,Clear.,9 km/h,52%,1012 mbar,16 km
694,20200229,22:51,2 °C,Clear.,13 km/h,52%,1012 mbar,16 km
